In [0]:
%python
from IPython.display import display, Markdown

display(Markdown("# MVP – Engenharia de Dados – Análise Econômica\n\nEste notebook apresenta o pipeline de dados construído para analisar a correlação entre IPCA, SELIC e IBOVESPA nos últimos 15 anos, com base em dados públicos simulados. O pipeline segue as etapas BRONZE → SILVER → GOLD, realiza modelagem em estrela e executa análises estatísticas."))


In [0]:
%python
# Configuração Inicial: Criar e usar o banco de dados para o projeto
spark.sql("CREATE DATABASE IF NOT EXISTS mvp_economia")
spark.sql("USE mvp_economia")


In [0]:
%python
from IPython.display import display, Markdown

display(Markdown("## Leitura dos Dados BRONZE\n\nNesta etapa, realizamos a leitura dos arquivos CSV que contêm os dados brutos de IPCA, SELIC e IBOVESPA."))


In [0]:
%python
path_ipca = "/FileStore/tables/ipca.csv"
path_selic = "/FileStore/tables/selic.csv"
path_ibov = "/FileStore/tables/ibov.csv"

df_ipca_bronze = spark.read.option("header", True).option("inferSchema", True).csv(path_ipca)
df_selic_bronze = spark.read.option("header", True).option("inferSchema", True).csv(path_selic)
df_ibov_bronze = spark.read.option("header", True).option("inferSchema", True).csv(path_ibov)


In [0]:
%python
display(dbutils.fs.ls("/FileStore/"))


In [0]:
%python
# Lendo o arquivo combinado BRONZE
path_economia = "/FileStore/economia_correlacao.csv"

df_economia_bronze = spark.read.option("header", True).option("inferSchema", True).csv(path_economia)

# Mostrar os dados lidos
display(df_economia_bronze)


In [0]:
%python
from IPython.display import display, Markdown

display(Markdown("## Transformação SILVER\n\nNesta etapa, realizamos a limpeza e padronização dos dados brutos, garantindo qualidade e consistência para as próximas análises."))


In [0]:
%python
# Removendo registros nulos
df_economia_silver = df_economia_bronze.dropna()

# Conferindo o resultado
display(df_economia_silver)


In [0]:
%python
# Salvando o dado como tabela GOLD
df_economia_silver.write.format("delta").mode("overwrite").saveAsTable("gold_economia")


In [0]:
%python
display(Markdown("## Modelagem em Estrela\n\nNesta etapa, criamos uma tabela fato principal e tabelas de dimensão para os indicadores econômicos, utilizando a camada GOLD como fonte de dados."))


In [0]:
%python
# Lendo a tabela GOLD
df_gold = spark.table("gold_economia")

# Criando Tabela Fato (com todas as colunas)
df_fato = df_gold.select(
    "data",
    "ipca",
    "selic",
    "ibov"
)

df_fato.write.format("delta").mode("overwrite").saveAsTable("fato_economia")

# Criando Tabela Dimensão IPCA
df_dim_ipca = df_gold.select(
    "data",
    "ipca"
)

df_dim_ipca.write.format("delta").mode("overwrite").saveAsTable("dim_ipca")

# Criando Tabela Dimensão SELIC
df_dim_selic = df_gold.select(
    "data",
    "selic"
)

df_dim_selic.write.format("delta").mode("overwrite").saveAsTable("dim_selic")

# Criando Tabela Dimensão IBOV
df_dim_ibov = df_gold.select(
    "data",
    "ibov"
)

df_dim_ibov.write.format("delta").mode("overwrite").saveAsTable("dim_ibov")


In [0]:
%python
spark.sql("DROP TABLE IF EXISTS fato_economia")
spark.sql("DROP TABLE IF EXISTS dim_ipca")
spark.sql("DROP TABLE IF EXISTS dim_selic")
spark.sql("DROP TABLE IF EXISTS dim_ibov")


In [0]:
%python
# Lendo a tabela GOLD
df_gold = spark.table("gold_economia")

# Criando Tabela Fato
df_fato = df_gold.select(
    "data",
    "ipca",
    "selic",
    "ibov"
)
df_fato.write.format("delta").mode("overwrite").saveAsTable("fato_economia")

# Criando Tabela Dimensão IPCA
df_dim_ipca = df_gold.select(
    "data",
    "ipca"
)
df_dim_ipca.write.format("delta").mode("overwrite").saveAsTable("dim_ipca")

# Criando Tabela Dimensão SELIC
df_dim_selic = df_gold.select(
    "data",
    "selic"
)
df_dim_selic.write.format("delta").mode("overwrite").saveAsTable("dim_selic")

# Criando Tabela Dimensão IBOV
df_dim_ibov = df_gold.select(
    "data",
    "ibov"
)
df_dim_ibov.write.format("delta").mode("overwrite").saveAsTable("dim_ibov")


In [0]:
%python
# Lendo a tabela GOLD
df_gold = spark.table("gold_economia")

# Criando Tabela Fato
df_fato = df_gold.select(
    "data",
    "ipca",
    "selic",
    "ibov"
)
df_fato.write.option("mergeSchema", "true").format("delta").mode("overwrite").saveAsTable("fato_economia")

# Criando Tabela Dimensão IPCA
df_dim_ipca = df_gold.select(
    "data",
    "ipca"
)
df_dim_ipca.write.option("mergeSchema", "true").format("delta").mode("overwrite").saveAsTable("dim_ipca")

# Criando Tabela Dimensão SELIC
df_dim_selic = df_gold.select(
    "data",
    "selic"
)
df_dim_selic.write.option("mergeSchema", "true").format("delta").mode("overwrite").saveAsTable("dim_selic")

# Criando Tabela Dimensão IBOV
df_dim_ibov = df_gold.select(
    "data",
    "ibov"
)
df_dim_ibov.write.option("mergeSchema", "true").format("delta").mode("overwrite").saveAsTable("dim_ibov")


In [0]:
from IPython.display import display, Markdown

display(Markdown("## Consultas SQL com JOINs\n\nNesta etapa, realizamos consultas relacionando a tabela fato com as tabelas dimensão, utilizando comandos SQL."))


In [0]:
# Exemplo de JOIN entre fato_economia e dim_ipca
display(spark.sql("""
SELECT f.data, f.selic, f.ibov, d.ipca
FROM fato_economia f
INNER JOIN dim_ipca d
ON f.data = d.data
ORDER BY f.data
"""))


In [0]:
# JOIN usando o nome do banco explicitamente
display(spark.sql("""
SELECT f.data, f.selic, f.ibov, d.ipca
FROM mvp_economia.fato_economia f
INNER JOIN mvp_economia.dim_ipca d
ON f.data = d.data
ORDER BY f.data
"""))


In [0]:
display(Markdown("### Consulta: Fato + Dimensão SELIC"))

display(spark.sql("""
SELECT f.data, f.ipca, f.ibov, d.selic
FROM mvp_economia.fato_economia f
INNER JOIN mvp_economia.dim_selic d
ON f.data = d.data
ORDER BY f.data
"""))


In [0]:
display(Markdown("### Consulta: Fato + Dimensão IBOV"))

display(spark.sql("""
SELECT f.data, f.ipca, f.selic, d.ibov
FROM mvp_economia.fato_economia f
INNER JOIN mvp_economia.dim_ibov d
ON f.data = d.data
ORDER BY f.data
"""))


In [0]:
from IPython.display import display, Markdown

display(Markdown("## Análises Estatísticas e Visualizações\n\nNesta etapa, analisamos a correlação entre IPCA, SELIC e IBOVESPA, aplicamos regressão linear e geramos gráficos para melhor compreensão dos dados."))


In [0]:
# Lendo a tabela GOLD para análise (com nome do banco)
df_gold = spark.table("mvp_economia.gold_economia")


In [0]:
# Importando a biblioteca pandas
import pandas as pd

# Convertendo Spark DataFrame para Pandas DataFrame
pdf_gold = df_gold.toPandas()

# Calculando a matriz de correlação
correlation_matrix = pdf_gold.corr()

# Exibindo a matriz de correlação
display(correlation_matrix)


In [0]:
from IPython.display import display, Markdown

display(Markdown("## Regressão Linear Simples\n\nNesta etapa, aplicamos uma regressão linear para analisar a relação entre a taxa SELIC e o índice IBOVESPA."))


In [0]:
# Importando biblioteca de regressão
from sklearn.linear_model import LinearRegression

# Definindo variável independente (X) e dependente (y)
X = pdf_gold[["selic"]]
y = pdf_gold["ibov"]

# Criando o modelo
model = LinearRegression()

# Treinando o modelo
model.fit(X, y)

# Coeficiente e Intercepto
coef = model.coef_[0]
intercept = model.intercept_

print(f"Equação da Regressão: IBOV = {coef:.2f} * SELIC + {intercept:.2f}")


In [0]:
# Importando a biblioteca de plotagem
import matplotlib.pyplot as plt

# Criando o gráfico de dispersão
plt.scatter(X.values, y.values, label="Dados reais")
plt.plot(X.values, model.predict(X), label="Regressão Linear", linewidth=2)

# Adicionando título e legendas
plt.title("Regressão Linear - SELIC vs IBOVESPA")
plt.xlabel("SELIC (%)")
plt.ylabel("IBOVESPA")
plt.legend()
plt.grid(True)
plt.show()


In [0]:
from IPython.display import display, Markdown

display(Markdown("## Conclusão\n\nO pipeline de dados implementado neste projeto permitiu a análise da relação entre o IPCA, a SELIC e o IBOVESPA nos últimos 15 anos. A estrutura em camadas (BRONZE, SILVER e GOLD) garantiu a qualidade e a organização dos dados.\n\nA análise de correlação revelou relações econômicas esperadas, como a correlação negativa entre a taxa SELIC e o índice IBOVESPA. A regressão linear simples confirmou essa tendência, mostrando que o aumento da SELIC tende a impactar negativamente o IBOVESPA.\n\nAs tabelas foram modeladas em estrela para facilitar consultas e análises, e o pipeline está preparado para ser reutilizado com novos dados futuros.")) 


In [0]:
df_gold = spark.table("mvp_economia.gold_economia")
display(df_gold)


In [0]:
%sql
SELECT f.data, f.selic, f.ibov, d.ipca
FROM mvp_economia.fato_economia f
INNER JOIN mvp_economia.dim_ipca d
ON f.data = d.data
ORDER BY f.data


In [0]:
correlation_matrix = pdf_gold.corr()
display(correlation_matrix)


In [0]:
plt.scatter(X.values, y.values, label="Dados reais")
plt.plot(X.values, model.predict(X), label="Regressão Linear", linewidth=2)
plt.title("Regressão Linear - SELIC vs IBOVESPA")


# 📚 Catálogo de Dados e Linhagem de Dados (Camada GOLD)

## 📄 Catálogo de Dados

| Nome da Tabela | Nome da Coluna | Tipo de Dado | Descrição | Origem |
|:--------------|:---------------|:------------|:---------|:------|
| fato_economia | data_id         | Date        | Data do registro econômico | Derivado da camada Silver |
| fato_economia | ipca_id         | Integer     | Chave estrangeira para dimensão IPCA | Derivado da camada Silver |
| fato_economia | selic_id        | Integer     | Chave estrangeira para dimensão SELIC | Derivado da camada Silver |
| fato_economia | ibov_id         | Integer     | Chave estrangeira para dimensão IBOVESPA | Derivado da camada Silver |
| dim_ipca      | ipca_id         | Integer     | Identificador único da dimensão IPCA | Gerado na camada GOLD |
| dim_ipca      | ipca_valor      | Decimal     | Valor mensal do IPCA | Fonte: CSV IPCA |
| dim_selic     | selic_id        | Integer     | Identificador único da dimensão SELIC | Gerado na camada GOLD |
| dim_selic     | selic_valor     | Decimal     | Valor mensal da taxa SELIC | Fonte: CSV SELIC |
| dim_ibov      | ibov_id         | Integer     | Identificador único da dimensão IBOVESPA | Gerado na camada GOLD |
| dim_ibov      | ibov_valor      | Decimal     | Valor mensal do índice IBOVESPA | Fonte: CSV IBOVESPA |

---

## 🛤️ Linhagem de Dados

**Descrição da Jornada dos Dados:**

1. **Coleta**:  
   - Dados brutos de IPCA, SELIC e IBOVESPA são carregados em arquivos CSV.

2. **Camada BRONZE**:  
   - Armazenamento inicial dos dados sem transformação no Databricks.

3. **Camada SILVER**:  
   - Limpeza, padronização e tratamento dos dados.

4. **Camada GOLD**:  
   - Modelagem dimensional em tabelas fato e dimensão, relacionando dados econômicos.

**Fluxo de Dados:**

